In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
display_table('RequestAccepted')

+--------------+-------------+-------------+
| requester_id | accepter_id | accept_date |
+--------------+-------------+-------------+
|      1       |      2      |  2016-06-03 |
|      1       |      3      |  2016-06-08 |
|      2       |      3      |  2016-06-08 |
|      3       |      4      |  2016-06-09 |
+--------------+-------------+-------------+


### Write an SQL query to find the people who have the most friends and the most friends number.

#### *The test cases are generated so that only one person has the most friends.The query result format is in the following example.*

```
+----+-----+
| id | num |
+----+-----+
| 3  | 3   |
+----+-----+
Explanation: 
The person with id 3 is a friend of people 1, 2, and 4, so he has three friends in total, which is the most number than any others.
```

In [3]:
%%sql 

SELECT requester_id 
FROM RequestAccepted

UNION ALL 

SELECT accepter_id 
FROM RequestAccepted

requester_id
1
1
2
3
2
3
3
4


In [4]:
%%sql 

SELECT 
    union_table.id AS id,
    COUNT(union_table.id) AS num 
FROM (
    SELECT requester_id AS id
    FROM RequestAccepted
    UNION ALL 
    SELECT accepter_id AS id
    FROM RequestAccepted
) AS union_table
GROUP BY id 

id,num
1,2
2,2
3,3
4,1


In [5]:
%%sql 

SELECT 
    union_table.id AS id,
    COUNT(union_table.id) AS num 
FROM (
    SELECT requester_id AS id
    FROM RequestAccepted
    UNION ALL 
    SELECT accepter_id AS id
    FROM RequestAccepted
) AS union_table
GROUP BY id 
ORDER BY num DESC

id,num
3,3
1,2
2,2
4,1


In [6]:
%%sql 

SELECT 
    union_table.id AS id,
    COUNT(union_table.id) AS num 
FROM (
    SELECT requester_id AS id
    FROM RequestAccepted
    UNION ALL 
    SELECT accepter_id AS id
    FROM RequestAccepted
) AS union_table
GROUP BY id 
ORDER BY num DESC
LIMIT 1;

id,num
3,3


# Using Pandas

In [7]:
import pandas as pd 

In [8]:
request_query = %sql SELECT * FROM RequestAccepted # type: ignore 
request_df = request_query.DataFrame()

display(request_df)

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [9]:
union_df = pd.concat([request_df['requester_id'], request_df['accepter_id']])
union_df

0    1
1    1
2    2
3    3
0    2
1    3
2    3
3    4
dtype: int64

In [10]:
union_df.value_counts()

3    3
1    2
2    2
4    1
Name: count, dtype: int64

In [11]:
final_df = union_df.value_counts().reset_index()
final_df

,index,count
0,3,3
1,1,2
2,2,2
3,4,1


In [12]:
final_df.columns = ['id', 'num']
final_df

,id,num
0,3,3
1,1,2
2,2,2
3,4,1


```py 
(method) def nlargest(
    n: int,
    columns: _str | list[_str],
    keep: NaPosition | Literal['all'] = ...
) -> DataFrame
```
- Return the first n rows ordered by columns in descending order.

- Return the first n rows with the largest values in columns, in descending order. The columns that are not specified are returned as well, but not used for ordering.

- This method is equivalent to `df.sort_values(columns, ascending=False).head(n)`, but more performant.

In [13]:
final_df.nlargest(1, 'num')

,id,num
0,3,3
